# Pré-processamento de dados
## Importando bibliotecas

In [1]:
from itertools import product
import string

import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.corpus import stopwords
from spacy.lang.pt.examples import sentences
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit, cross_validate

from src.data import preprocessing


nltk.download('rslp')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/tayna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/tayna/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /home/tayna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tayna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Carregamento dos Dados

In [2]:
output_path = "../data/interim/news.csv"
df_corpus = pd.read_csv(output_path)

## Pré-processamento 

In [3]:
df_corpus = (
    df_corpus
    .assign(
        clean_text = df_corpus['text']
        .apply(str.lower) # coloca texto todo para minúscula
        .apply(preprocessing.remove_date) # remove datas dd/mm/YYYY
        .apply(preprocessing.remove_date, style='diamesextensoano') # remove datas com mês escrito por extenso
        .apply(preprocessing.remove_weekday) # remove dias da semana
        .apply(preprocessing.remove_linebreaks) # remove quebras de linha
        .apply(preprocessing.remove_tabs) # remove tabs
        .apply(preprocessing.symbol_remove) # remove caracteres especiais exceto o hífen
    )
)
df_corpus.sample(5)

,text,label,clean_text
3209,A Polícia Federal pediu em outubro de 2015 a ...,true,a polícia federal pediu em outubro de 2015 a ...
6007,"Embaixador da Coreia do Norte em Londres: ""Não...",fake,embaixador da coreia do norte em londres não t...
2172,Morte brutal de universitário em assalto choca...,true,morte brutal de universitário em assalto choca...
6867,"É o fim! No maior estado do país, 30% dos pre...",fake,é o fim no maior estado do país 30 dos prefei...
1465,JBS vende para a Minerva negócios na Argentina...,true,jbs vende para a minerva negócios na argentina...


## Processamento

In [4]:
df_corpus = (
    df_corpus
    .assign(
        tokens = df_corpus['clean_text'].apply(preprocessing.tokenize), # cria lista de palavras de cada texto
    )
)
df_corpus.sample(5)

,text,label,clean_text,tokens
5581,"Filho de Teori: ""Se eles compram o silêncio, p...",fake,filho de teori se eles compram o silêncio pode...,"[filho, de, teori, se, eles, compram, o, silên..."
3914,Número de inquéritos contra Lula já chegam a 1...,fake,número de inquéritos contra lula já chegam a 1...,"[número, de, inquéritos, contra, lula, já, che..."
5371,"Ex-presidente da Andrade Gutierrez confirma: ""...",fake,ex-presidente da andrade gutierrez confirma di...,"[ex-presidente, da, andrade, gutierrez, confir..."
1144,Prefeitura de SP flexibiliza futuras regras pa...,true,prefeitura de sp flexibiliza futuras regras pa...,"[prefeitura, de, sp, flexibiliza, futuras, reg..."
1536,"Lula foi levado pela PF porque foi delatado, d...",true,lula foi levado pela pf porque foi delatado di...,"[lula, foi, levado, pela, pf, porque, foi, del..."


In [5]:
df_corpus = (
    df_corpus
    .assign(
        no_stopwords = df_corpus.tokens.apply(preprocessing.stopwords_remove), # remove palavras comuns de baixo valor semântico
        lemmatization_plus_stop_words = df_corpus.tokens.apply(preprocessing.lemmatization), # 
        stemming_plus_stopwords = df_corpus.tokens.apply(preprocessing.stemming) #         
    )
)
df_corpus.sample(5)

,text,label,clean_text,tokens,no_stopwords,lemmatization_plus_stop_words,stemming_plus_stopwords
493,O magnata que doou fortuna de US$ 8 bilhões em...,true,o magnata que doou fortuna de us 8 bilhões em ...,"[o, magnata, que, doou, fortuna, de, us, 8, bi...","(magnata, doou, fortuna, us, 8, bilhões, segre...",o magnata que doar fortuna de us 8 bilhão em s...,o magnat que doou fortun de us 8 bilhã em segr...
830,PF prende Wesley Batista em SP e cumpre nova p...,true,pf prende wesley batista em sp e cumpre nova p...,"[pf, prende, wesley, batista, em, sp, e, cumpr...","(pf, prende, wesley, batista, sp, cumpre, nova...",pf prender Wesley Batista em sp e cumprir novo...,pf prend wesley batist em sp e cumpr nov pris ...
3667,SUICÍDIO POLÍTICO: O Congresso Brasileiro prov...,fake,suicídio político o congresso brasileiro provo...,"[suicídio, político, o, congresso, brasileiro,...","(suicídio, político, congresso, brasileiro, pr...",suicídio político o Congresso brasileiro prova...,suicídi polít o congress brasil prov ao pov qu...
4175,Dilma convida ministro do Supremo para jantar ...,fake,dilma convida ministro do supremo para jantar ...,"[dilma, convida, ministro, do, supremo, para, ...","(dilma, convida, ministro, supremo, jantar, re...",Dilma conver ministro de o Supremo para jantar...,dilm conv ministr do supr par jant e receb res...
1348,Menino filho de vice-prefeita faz vídeo selfie...,true,menino filho de vice-prefeita faz vídeo selfie...,"[menino, filho, de, vice-prefeita, faz, vídeo,...","(menino, filho, vice-prefeita, faz, vídeo, sel...",Menino filho de vice-prefeita fazer vídeo self...,menin filh de vice-prefeit faz víde selfi enqu...


In [9]:
df_corpus = (
    df_corpus
    .assign(
        lemmatization_wo_stopwords = df_corpus.no_stopwords.apply(preprocessing.lemmatization), # 
        stemming_wo_stopwords = df_corpus.no_stopwords.apply(preprocessing.stemming) #         
    )
)
df_corpus.sample(5)

,text,label,clean_text,tokens,no_stopwords,lemmatization_plus_stop_words,stemming_plus_stopwords,lemmatization_wo_stopwords,stemming_wo_stopwords
5409,A China acordou! Presidente Xi Jinping autoriz...,fake,a china acordou presidente xi jinping autoriza...,"[a, china, acordou, presidente, xi, jinping, a...","(china, acordou, presidente, xi, jinping, auto...",o China acordar presidente xi jinping autoriza...,a chin acord presid xi jinping autoriz plan es...,China acordar presidente xi jinping autorizar ...,chin acord presid xi jinping autoriz plan estr...
3444,"'O PSDB deve ter juízo', diz FHC\n\nPara o ex-...",true,o psdb deve ter juízo diz fhc para o ex-presi...,"[o, psdb, deve, ter, juízo, diz, fhc, para, o,...","(psdb, deve, ter, juízo, diz, fhc, ex-presiden...",o psdb dever ter juízo dizer fhc para o ex-pre...,o psdb dev ter juíz diz fhc par o ex-presid o ...,psdb dever ter juízo dizer fhc ex-presidente n...,psdb dev ter juíz diz fhc ex-presid nov comand...
3330,"Quarta-feira, 21 de fevereiro de 2018. Boa noi...",true,boa noite aqui estão as principais notícias ...,"[boa, noite, aqui, estão, as, principais, notí...","(boa, noite, aqui, principais, notícias, termi...",bom noite aqui estar o principal notícia para ...,boa noit aqu est as princip notíc par voc term...,bom noite aqui principal notícia terminar dia ...,boa noit aqu princip notíc termin dia bem-info...
5244,EBC corrige matéria do Diário do Brasil. Marid...,fake,ebc corrige matéria do diário do brasil marido...,"[ebc, corrige, matéria, do, diário, do, brasil...","(ebc, corrige, matéria, diário, brasil, marido...",ebc corrigir matéria de o Diário de o Brasil m...,ebc corrig matér do diári do brasil marid de m...,ebc corrigir matéria Diário Brasil marido mari...,ebc corrig matér diári brasil marid mariet sev...
2753,Saiba como o meio político reagiu à decisão do...,true,saiba como o meio político reagiu à decisão do...,"[saiba, como, o, meio, político, reagiu, à, de...","(saiba, meio, político, reagiu, decisão, tse, ...",saber como o meio político reagir a o decisão ...,saib com o mei polít reag à decis do tse sobr ...,saber meio político reagir decisão tse sobre c...,saib mei polít reag decis tse sobr chap dilma-...


## Salvando dados pré-processados

In [10]:
(
    df_corpus
    .drop(columns=["clean_text", "tokens", "no_stopwords"], axis=1)
    .to_csv("../data/processed/textos_pre-processados.csv", index=False)
)